In [1]:
# Imports
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

#### Load df

In [3]:
df = pd.read_csv("fremont_df.csv")
df

,Date,"Fremont Bridge Sidewalks, south of N 34th St Total","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk"
0,10/02/2012 01:00:00 PM,55.0,7.0,48.0
1,10/02/2012 02:00:00 PM,130.0,55.0,75.0
2,10/02/2012 03:00:00 PM,152.0,81.0,71.0
3,10/02/2012 04:00:00 PM,278.0,167.0,111.0
4,10/02/2012 05:00:00 PM,563.0,393.0,170.0
...,...,...,...,...
111691,06/30/2025 07:00:00 PM,NaN,NaN,NaN
111692,06/30/2025 08:00:00 PM,NaN,NaN,NaN
111693,06/30/2025 09:00:00 PM,NaN,NaN,NaN
111694,06/30/2025 10:00:00 PM,NaN,NaN,NaN


#### EDA

In [14]:
df.shape

(111696, 4)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111696 entries, 0 to 111695
Data columns (total 4 columns):
 #   Column                                                              Non-Null Count   Dtype  
---  ------                                                              --------------   -----  
 0   Date                                                                111696 non-null  object 
 1   Fremont Bridge Sidewalks, south of N 34th St Total                  111596 non-null  float64
 2   Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk  111596 non-null  float64
 3   Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk  111596 non-null  float64
dtypes: float64(3), object(1)
memory usage: 3.4+ MB
